In [1]:
# Imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

%matplotlib inline 

In [2]:
# That Data
df = pd.read_json('../../../Data/data.json')

In [8]:
import requests
api_url='https://hxobin8em5.execute-api.us-west-2.amazonaws.com/api/'
api_key='vYm9mTUuspeyAWH1v-acfoTlck-tCxwTw9YfCynC'

def get_data(next_sequence_number):
    """Fetch data from the API."""
    payload = {'api_key': api_key,
               'sequence_number': next_sequence_number}
    response = requests.post(api_url, json=payload)
    data = response.json()
    next_sequence_number = data['_next_sequence_number']
    return data['data']

In [9]:
x = get_data(0)[0]

In [36]:
import math 

def missing_data(x):
    '''
    Param:
        x: dictionary
    Return:
        1 if missing value is found else 0
    '''
    for value in x.values():
        if value == '':
            return 1
    return 0

def no_previous_payout(x):
    '''
    Param:
        list of previous payouts 
    '''
    return 1 if len(x) <= 0 else 0

def payout_name_flag(x):
    '''
    Param:
        list of previous payouts 
    '''
    for d in x:
        if len(d['name']) < 3:
            return 1
    return 0

def payout_toself(payee, payouts):
    '''
    Param:
        payee: payee 
        payouts: list of previous payouts 
    '''
    for pay in payouts:
        if payee.lower() in pay['name'].lower():
            return 1
    return 0

def payee_name_flag(x):
    return 1 if len(x) < 3 else 0

def delivery(x):
    if math.isnan(x): return 0
    return int(x)

In [45]:
def parse_data(data):
    '''
        Param:
            api data call
    '''
    temp = dict()
    
    if type(data) == type(dict()):
        temp = {
            'payout_toself': payout_toself(data['payee_name'], data['previous_payouts']),
            'missing_data': missing_data(data),
            'previous_payout': no_previous_payout(data['previous_payouts']),
            'no_payout_name': payout_name_flag(data['previous_payouts']),
            'no_payee_name': payee_name_flag(data['payee_name']),
            'account_type': data['user_type'],
            'delivery_method': delivery(data['delivery_method']),
            'user_age': data['user_age']}
    else:
        temp = {
            'payout_toself': data.apply(lambda x: payout_toself(x.payee_name, x.previous_payouts), axis = 1),
            'missing_data': data.isnull().any(axis = 1) * 1,
            'previous_payout': data.previous_payouts.apply(lambda x: no_previous_payout(x)),
            'no_payout_name': data.previous_payouts.apply(lambda x: payout_name_flag(x)),
            'no_payee_name': data.payee_name.apply(lambda x: payee_name_flag(x)),
            'account_type': data.user_type,
            'delivery_method': data.delivery_method.apply(lambda x: delivery(x)),
            'user_age': data.user_age }
    
    return temp

In [43]:
parse_data(x)

{'payout_toself': 1,
 'missing_data': 1,
 'previous_payout': 0,
 'no_payout_name': 1,
 'no_payee_name': 1,
 'account_type': 3,
 'delivery_method': 1,
 'user_age': 554}

In [44]:
parse_data(df)

{'payout_toself': 0        0
 1        1
 2        1
 3        1
 4        1
         ..
 14332    0
 14333    1
 14334    1
 14335    1
 14336    0
 Length: 14337, dtype: int64,
 'missing_data': 0        0
 1        0
 2        1
 3        1
 4        0
         ..
 14332    1
 14333    0
 14334    0
 14335    0
 14336    1
 Length: 14337, dtype: int64,
 'previous_payout': 0        1
 1        0
 2        0
 3        0
 4        0
         ..
 14332    1
 14333    0
 14334    0
 14335    0
 14336    1
 Name: previous_payouts, Length: 14337, dtype: int64,
 'no_payout_name': 0        0
 1        0
 2        0
 3        1
 4        1
         ..
 14332    0
 14333    0
 14334    1
 14335    1
 14336    0
 Name: previous_payouts, Length: 14337, dtype: int64,
 'no_payee_name': 0        1
 1        0
 2        0
 3        1
 4        0
         ..
 14332    1
 14333    0
 14334    1
 14335    1
 14336    1
 Name: payee_name, Length: 14337, dtype: int64,
 'account_type': 0        1
 1       

In [ ]:
df